In [1]:
%load_ext lab_black

# Get a 'detector' object

In [2]:
from pyxel.detectors import CCD, CCDCharacteristics, CCDGeometry, Environment, Material
import numpy as np
import typing as t
from copy import deepcopy


def create_detector() -> CCD:
    """Create a valid `CCD` object."""
    pixel_2d = np.zeros((100, 10), dtype=int)

    pixel_2d[25:50, :] = 100
    pixel_2d[75:, :] = 100

    detector = CCD(
        geometry=CCDGeometry(row=100, col=10),
        material=Material(),
        environment=Environment(),
        characteristics=CCDCharacteristics(fwc=100_000),
    )

    detector.pixel.array = pixel_2d

    return detector

In [3]:
detector = create_detector()

detector

In [4]:
%%timeit

_ = deepcopy(detector)

170 µs ± 5.67 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Define 1 trap and 5 traps

In [5]:
one_trap = [{"density": 90.0, "release_timescale": 1.2}]

five_traps = [
    {"density": 91.98081597, "release_timescale": 2.07392977},
    {"density": 100.32049957, "release_timescale": 0.75635299},
    {"density": 103.70445648, "release_timescale": 1.48364189},
    {"density": 100.76309597, "release_timescale": 0.70015936},
    {"density": 104.31871946, "release_timescale": 1.30312337},
]

# Test with Arctic Vanilla

First version using ``arctic`` library from PyPI

## Run for 1 trap

In [6]:
from pyxel.models.charge_transfer.arctic_cti.models_arctic_vanilla import arctic_add

In [7]:
%%time

detector_vanilla_1_trap = deepcopy(detector)

arctic_add(
    detector=detector_vanilla_1_trap,
    well_fill_power=0.8,
    instant_traps=one_trap,
    express=0,
)

CPU times: user 13.2 s, sys: 48.6 ms, total: 13.3 s
Wall time: 13.3 s


In [15]:
%%timeit

# Done in 13.7 s ± 822 ms
arctic_add(
    detector=deepcopy(detector),
    well_fill_power=0.8,
    instant_traps=one_trap,
    express=0,
)

13.7 s ± 822 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Run for 5 traps

In [8]:
%%time

detector_vanilla_5_traps = deepcopy(detector)

arctic_add(
    detector=detector_vanilla_5_traps,
    well_fill_power=0.8,
    instant_traps=five_traps,
    express=0,
)

CPU times: user 16.7 s, sys: 63.8 ms, total: 16.8 s
Wall time: 16.9 s


In [16]:
%%timeit

# Done in 15.9 s ± 411 ms
arctic_add(
    detector=deepcopy(detector),
    well_fill_power=0.8,
    instant_traps=five_traps,
    express=0,
)

15.9 s ± 411 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Arctic with numba

First version with a modified `arctic` library

## Run for 1 trap

In [9]:
from pyxel.models.charge_transfer.arctic_cti.models_arctic_numba import arctic_add_numba

In [10]:
%%time

detector_fast_1_trap = deepcopy(detector)

arctic_add_numba(
    detector=detector_fast_1_trap,
    well_fill_power=0.8,
    instant_traps=one_trap,
    express=0,
)

CPU times: user 33.2 s, sys: 290 ms, total: 33.5 s
Wall time: 33.7 s


In [17]:
%%timeit

#. Done in 1.24 s ± 447 ms
arctic_add_numba(
    detector=deepcopy(detector),
    well_fill_power=0.8,
    instant_traps=one_trap,
    express=0,
)

1.24 s ± 447 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Run for 5 traps

In [11]:
%%time

detector_fast_5_traps = deepcopy(detector)

arctic_add_numba(
    detector=detector_fast_5_traps,
    well_fill_power=0.8,
    instant_traps=five_traps,
    express=0,
)

CPU times: user 2.01 s, sys: 15.7 ms, total: 2.03 s
Wall time: 2.02 s


In [21]:
%%timeit

# Done in 1.98 s ± 75.7 ms
arctic_add_numba(
    detector=deepcopy(detector),
    well_fill_power=0.8,
    instant_traps=five_traps,
    express=0,
)

1.69 s ± 45.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Comparison with vanilla arctic

In [14]:
# Compare results from vanilla 'arctic' and this 'arctic'
np.testing.assert_almost_equal(
    detector_vanilla_1_trap.pixel.array, detector_fast_1_trap.pixel.array
)
np.testing.assert_almost_equal(
    detector_vanilla_5_traps.pixel.array, detector_fast_5_traps.pixel.array
)

**Comparison with 1 Trap**


| Model           |  Time   | Speedup |
| --------------- |:-------:| -------:| 
| Vanilla Arctic  | 13.7 s  |  11x    |
| Numba Arctic    | 1.24 s  |  1x     |


**Comparison with 5 Traps**


| Model           |  Time  | Speedup |
| --------------- |:------:| -------:| 
| Vanilla Arctic  | 15.9 s |  9.4x   |
| Numba Arctic    | 1.69 s |  1x     |